<a href="https://colab.research.google.com/github/Alton1998/LLM-Detect-AI-Generated-Text/blob/main/LLM_LLM_Generated_Text_Detection_Pretrained_MultinomialNaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import string
import time
import nltk
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [ ]:
!pip install datasets
nltk.download("stopwords")

In [ ]:
from datasets import load_dataset

In [ ]:
deep_fake_text_data_set_train = load_dataset("yaful/DeepfakeTextDetect", split="train")
deep_fake_text_data_set_val = load_dataset("yaful/DeepfakeTextDetect", split="validation")
deep_fake_text_data_set_test = load_dataset("yaful/DeepfakeTextDetect", split="test")

In [ ]:
deep_fake_text_data_set_train_df = deep_fake_text_data_set_train.to_pandas()
deep_fake_text_data_set_val_df = deep_fake_text_data_set_val.to_pandas()
deep_fake_text_data_set_test_df = deep_fake_text_data_set_test.to_pandas()

def invert_label(x):
  if x==1:
    return 0
  elif x==0:
    return 1

In [ ]:
deep_fake_text_data_set_train_df.shape

In [ ]:
deep_fake_text_data_set_train_df["new label"] = deep_fake_text_data_set_train_df['label'].apply(invert_label)
deep_fake_text_data_set_val_df["new label"] =  deep_fake_text_data_set_val_df['label'].apply(invert_label)
deep_fake_text_data_set_test_df["new label"] = deep_fake_text_data_set_test_df['label'].apply(invert_label)

In [ ]:
deep_fake_text_data_set_train_df['length'] = deep_fake_text_data_set_train_df['text'].apply(len)
deep_fake_text_data_set_val_df['length'] = deep_fake_text_data_set_val_df['text'].apply(len)
deep_fake_text_data_set_test_df['length'] = deep_fake_text_data_set_test_df['text'].apply(len)

In [ ]:
def text_process(mess):
    """
    1.remove punc
    2. remove stop words
    3. return list of clean text words
    """
    nopunc = [char for char in mess if char not in string.punctuation]
    nopunc = "".join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [ ]:
text_train,text_test,label_train,label_test = train_test_split(deep_fake_text_data_set_train_df['text'],deep_fake_text_data_set_train_df['new label'],test_size=0.3,random_state=101)

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
pipeline = Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf',TfidfTransformer()),
    ('classifier',MultinomialNB())
])

In [ ]:
start_time = time.time()
pipeline.fit(text_train,label_train)
print(f'\nDuration: {time.time() - start_time:.0f} seconds')

In [ ]:
predictions = pipeline.predict(text_test)

In [ ]:
print(classification_report(label_test,predictions))